#download data set

In [4]:
import pandas as pd
import numpy as np

In [7]:
url = 'https://raw.githubusercontent.com/NLPforLRLsProjects/SAfriSenti-Corpus/refs/heads/main/setswana_tweets.csv'
df_setswana_tweets = pd.read_csv(url)
print("Number of records: ",df_setswana_tweets.shape[0])

Number of records:  3000


#tokenize dataset

#Create neural model

#evaluate model